In [10]:
# install tensorflow
%pip install tensorflow
%pip install pandas
%pip install numpy
%pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 8.0 MB/s eta 0:00:00MB/s eta 0:00:01:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 9.4 MB/s eta 0:00:00 MB/s eta 0:00:01:01m
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [15]:
# monte um dataset com os arquivos na pasta datasets/dogs e datasets/cats
# no dataset deve ter 2 colunas: filename e label
# o label deve ser 0 para gatos e 1 para cachorros
# o dataset deve ser dividido em 80% para treino e 20% para teste
# o dataset deve ser montado em um dataframe pandas

import pandas as pd
import numpy as np
import os

# monta o dataset
def mount_dataset():
    dataset = []
    for label in ['cats', 'dogs']:
        for filename in os.listdir(f'datasets/{label}'):
            dataset.append({'filename': filename, 'label': label, 'value': 0 if label == 'cats' else 1})
    return pd.DataFrame(dataset)

# monta o dataset
df = mount_dataset()
df.head()


,filename,label,value
0,cat_190.jpg,cats,0
1,cat_147.jpg,cats,0
2,cat_542.jpg,cats,0
3,cat_595.jpg,cats,0
4,cat_422.jpg,cats,0


In [16]:
# embaralha o dataset
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,filename,label,value
0,dog_398.jpg,dogs,1
1,cat_96.jpg,cats,0
2,dog_43.jpg,dogs,1
3,dog_219.jpg,dogs,1
4,dog_141.jpg,dogs,1


In [17]:

# divide o dataset em treino e teste
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['filename'], df['value'], test_size=0.2, random_state=42)


In [18]:

# monta o modelo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/Users/clairtonluz/projects/personal/MBA-em-ciencia-de-dados/Deep Leaning/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:

# treina o modelo
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_dataframe(
    df,
    directory='datasets/',
    x_col='filename',
    y_col='value',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary')

model.fit(train_generator, epochs=10)

TypeError: If class_mode="binary", y_col="value" column values must be strings.

In [ ]:

# avalia o modelo
test_generator = test_datagen.flow_from_dataframe(
    df,
    directory='datasets',
    x_col='filename',
    y_col='label',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

model.evaluate(test_generator)